<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Logo_DuocUC.svg/2560px-Logo_DuocUC.svg.png' width=50%, height=20%>

# INTRODUCCION

Para desarrollar este laboratorio, usaremos el set de datos Credit Card Fraut Detection de la web de Kaggle. Son 66 MB que al descomprimir ocuparán 150MB. Usaremos el archivo creditcard.csv. Este dataset consta de 285.000 filas con 31 columnas (features). Como la información es privada, no sabemos realmente que significan los features y están nombradas como V1, V2, V3, etc. excepto por las columnas Time y Amount (el importe de la transacción). Y nuestras clases son 0 y 1 correspondiendo con “transacción Normal” ó “Hubo Fraude”.

En este ejercicio compararemos una Regresión Logística contra Random Forrest.

NOTA : dado el tamaño del dataset, el proceso de carga puede ser lento.

## Importar librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier

from pylab import rcParams

from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedBaggingClassifier

from collections import Counter

#set up graphic style in this case I am using the color scheme from xkcd.com
rcParams['figure.figsize'] = 14, 8.7 # Golden Mean
LABELS = ["Normal","Fraud"]
#col_list = ["cerulean","scarlet"]# https://xkcd.com/color/rgb/
#sns.set(style='white', font_scale=1.75, palette=sns.xkcd_palette(col_list))

%matplotlib inline

## Cargamos Datos

En este caso subiremos podrás ubicar el archivo comprimido en la dirección indicada. Debes dejarlo local para leerlo ejecutando lo siguiente:

In [ ]:
# Descargar desde https://www.kaggle.com/mlg-ulb/creditcardfraud/data

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    name=fn
df = pd.read_csv(name, sep=",")
#df = pd.read_csv("creditcard.csv")
df.head(n=5)

In [ ]:
df.shape

## Vemos Desbalanceo

**Revisa cuantas observaciones corresponden a cada clase.**

In [ ]:
pd.value_counts(df['Class'], sort = True) #class comparison 0=Normal 1=Fraud

In [ ]:
normal_df = df[df.Class == 0] #registros normales
fraud_df = df[df.Class == 1] #casos de fraude

## Creamos Dataset

In [ ]:
y = df['Class']
X = df.drop('Class', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [ ]:
def mostrar_resultados(y_test, pred_y):
    conf_matrix = confusion_matrix(y_test, pred_y)
    plt.figure(figsize=(8, 8))
    sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
    plt.title("Confusion matrix")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.show()
    print (classification_report(y_test, pred_y))

## Ejecutamos Modelo con LogisticRegresion para poder Comparar

In [ ]:
def run_model_balanced(X_train, X_test, y_train, y_test):
    clf = LogisticRegression(C=1.0,penalty='l2',random_state=1,solver="newton-cg",class_weight="balanced")
    clf.fit(X_train, y_train)
    return clf

model = run_model_balanced(X_train, X_test, y_train, y_test)

### Veamos como responde en el test set

In [ ]:
pred_y = model.predict(X_test)
mostrar_resultados(y_test, pred_y)

# **ANALIZAR**

¿Cuál es el nivel de exactitud de este modelo?

¿Qué otras métricas puedes comentar? ¿Qué dicen?

# Probamos con Random Forest

## ATENCION: Este modelo toma algo más de tiempo en entrenar

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Crear el modelo con 100 arboles
model = RandomForestClassifier(n_estimators=100,
                               bootstrap = True,verbose=2,
                               max_features = 'sqrt')
# entrenar!
model.fit(X_train, y_train)

### Revisemos los resultados

In [ ]:
pred_y = model.predict(X_test)
mostrar_resultados(y_test, pred_y)

# **Analizar**

Lee las métricas anteriores, ¿qué puedes comentar de estas?

# Otro Bosque: Random Forest -más veloz-

In [ ]:
# otro modelo, variando hiperparámetros
model = RandomForestClassifier(n_estimators=100, class_weight="balanced",
                               max_features = 'sqrt', verbose=2, max_depth=6,
                               oob_score=True, random_state=50)
# a entrenar
model.fit(X_train, y_train)

## Veamos la Confusion Matrix con el conjunto de Test

In [ ]:
pred_y = model.predict(X_test)
mostrar_resultados(y_test, pred_y)

# **ANALIZAR**

En este modelo, ¿que reflejan las métricas?

# Comprobamos Resultados

## **Investiga**

¿Qué significa y para que sirve el "roc_auc_score"?

In [ ]:
from sklearn.metrics import roc_auc_score

# Calculate roc auc
roc_value = roc_auc_score(y_test, pred_y)

In [ ]:
print(roc_value)

0.9180763016207248


El valor de roc cuanto más cerca de 1, mejor. si fuera 0.5 daría igual que fuesen valores aleatorios y sería un mal modelo

# **CONCLUSIONES**

Reflexiona sobre este ejercicio.

¿Qué puedes comentar sobre Random Forrest?

Comparte tu análisis con el resto del curso.